In [1]:
import pandas as pd
import numpy as np
import requests

# Read the table from the Wikipedia site

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url, flavor = 'bs4')

# Define a function to arrange the dataset

In [3]:
def str_extract(x):
    if 'Not assigned' in x:
        postal_code = str(x)[:3]
        borough = neighbor = np.nan
    else:
        postal_code = str(x)[:3]
        bor_neig = str(x)[3:].split('(')
        borough = bor_neig[0]
        neighbor = bor_neig[1].replace(')', '').replace(' /', ',')
        if neighbor == '': neighbor = borough
        
    return [postal_code, borough, neighbor]

# Filter and create a structured dataset

In [4]:
data_stacked = data[0].applymap(str_extract).stack()
new_list = []
for x in data_stacked:
    new_list.append(x)
dataset = pd.DataFrame(new_list, columns = ['PostalCode', 'Borough', 'Neighborhood'])

In [5]:
dataset.head(12)

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
7,M8A,NaN,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [6]:
dataset.shape

(180, 3)

# Read the GeoSpatial Dataset

In [7]:
loc_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
location = pd.read_csv(loc_url, names = ['PostalCode', 'Latitude', 'Longitude'])

# Execute a inner join in both datasets

In [8]:
dataset = pd.merge(dataset, location, on = 'PostalCode')
dataset['Latitude'] = pd.to_numeric(dataset['Latitude'])
dataset['Longitude'] = pd.to_numeric(dataset['Longitude'])
dataset.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don MillsNorth,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
